In [2]:
import ee
import geemap
import geemap.colormaps
import ipywidgets as w
from IPython.display import display, display_html

In [3]:
import geemap.toolbar

cmp = geemap.colormaps.palettes

Map = geemap.Map(center=(-5,-64), zoom=5, #basemap = 'TERRAIN',
                 draw_ctrl=False, attribution_ctrl=False, data_ctrl=False)
Map.layout.height = 'auto'

# Select only toolbar controls "inspector[0]" and "change_basemap[8]"
Map.toolbar.children = (Map.toolbar.children[0], Map.toolbar.children[8], Map.toolbar.children[-2])
Map.toolbar.layout.grid_template_rows='32px'

photo = w.Image(value=open('../logos/Photos.jpg', 'rb').read(),
                    width='97%',
                    height='auto')

logo_layout = w.Layout(width='auto', height='100px')

logo_hge = w.Image(value=open('../logos/logo_hge.png', 'rb').read(), layout=logo_layout)
logo_iph = w.Image(value=open('../logos/logo_iph.png', 'rb').read(), layout=logo_layout)
logo_ufrgs = w.Image(value=open('../logos/logo_ufrgs.png', 'rb').read(), layout=logo_layout)
logo_ird = w.Image(value=open('../logos/logo_ird.png', 'rb').read(), layout=logo_layout)

logos = w.HBox([logo_hge, logo_iph, logo_ufrgs, logo_ird])

title = w.Image(value=open('../logos/logo_app.png', 'rb').read(),  layout=w.Layout(width='75%', height='auto'))

elements_width = '97%'
btnStyle = w.ButtonStyle(button_color='lightgreen')
btnLayout = w.Layout(width=elements_width)

formlink = "https://forms.gle/TjDQ9cdt8ocdSkRD9"
amzpol = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_3").filter(ee.Filter.eq('PFAF_ID', 622))

def show_map(map2show):
    try:
        Map.remove_colorbar()
    except:
        pass
    
    img = map2show[0].clip(ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_3").filter(ee.Filter.eq('PFAF_ID', 622)))
    vmin = map2show[1]
    vmax = map2show[2]
    mtype = map2show[3]
    
    
    if vmin>0:
        img = img.updateMask(img.gte(vmin)).rename('value')
    else:
        img = img.updateMask(img.gt(vmin)).rename('value')
    img = img.updateMask(img.lte(vmax)).rename('value')
    
    if mtype == 'binary':
        parvis = {'min': vmin, 'max': vmin, 'palette': cmp.viridis_r}
        img = img.where(img, 1)
        Map.addLayer(img, parvis, 'Inundation map')
    else:
        parvis = {'min': vmin, 'max': vmax, 'palette': cmp.viridis_r}
        Map.addLayer(img, parvis, 'Inundation map')
        Map.add_colorbar(parvis, label = None, discrete=False, orientation='vertical', layer_name = 'Inundation map')


In [4]:
# EN version

# Description
subtitle = w.HTML(value='<{size}>Displaying the inundation extent of the largest river basin on Earth <br/> WebGIS platform | Version 0.1</{size}>'.format(size='h3'))
description = w.HTML(value= 
'<h4 align="justify">The Amazon-Inundation WebGIS App allows the visualization of the inundation extent across the Amazon River basin '
         'based on 30 datasets that were developed in the last decades with different spatial and temporal resolutions. </h4>'
                     '<p style="font-size:12px;">Developed by Rafael Barbedo and Ayan Fleischmann <br/>'
                     'Institute of Hydraulic Research, Federal University of Rio Grande do Sul</p>')
description.layout.width = elements_width

about = w.HTML('<{size} align="justify">The Amazon River basin is the largest on Earth, and harbors unique freshwater ecosystems on which thousands of people rely for their livelihoods, '
                     'contrasting ecosystems are adapted to its seasonal inundation dynamics, and major biogeochemical processes occur. <br/>'
                     
'The Amazon-Inundation WebGIS App provides an easy way to visualize the inundation extent across the basin based on various datasets that were developed in the last decades. <br/>'
                     
'The products have different data sources, from passive microwave and synthetic aperture radar to optical imagery and hydrological modeling, with spatial resolution varying from 12.5 m to 25 km. <br/>'

'Four types of products are available: <br/>'
               '- Two 1 km agreement maps, showing, for each 1 km pixel, how many datasets agree that it is flooded considering the long-term maximum or minimum inundation maps. <br/>'
               '- Wetland hydrological types (1- interfluvial wetlands and small floodplains; 2- medium to large river floodplains). <br/>'
               '- Regional/local scale: products developed for specific sub-regions of the Amazon. <br/>'
               '- Basin scale: products developed for global scale or especifically for the Amazon River basin. <br/>'

'The “Regional/local scale” and “Basin scale” products are available as flood frequency or binary (wet/dry) inundation maps. '
'Long-term flood frequency maps are provided for the dynamic products, for which the higher the pixel value, the longer it is flooded throughout the hydrological cycle. <br/><br/>'

'References and data availability are provided for each dataset. For more details, and to cite this tool, please refer to: <br/><br/>'

'Fleischmann et al., 2021, How much inundation occurs in the Amazon River basin? '
'<a href="https://www.essoar.org/doi/10.1002/essoar.10508718.1" target="_blank">https://forms.gle/HcqZj94K55G6QSFy8</a></{size}>'
                     .format(size='p'))

# Categories
catList = ['Agreement among datasets at 1 km', 'Wetland hydrological types', 'Regional/local scale', 'Basin scale']
catW = w.widgets.RadioButtons(options=catList, description='Category: ', layout=w.Layout(width='400px'))

# Maps
mapList = {'Agreement among datasets at 1 km':
            {
            'Maximum extent agreement': [ee.Image('users/rbfontana/inundation_amazon/AgreementMax'), 1, 14, 'frequency',
'Products agreement on maximum inundation (number of products).'
'<br/><br/>Data acquisition: <a href="https://forms.gle/HcqZj94K55G6QSFy8" target="_blank">https://forms.gle/HcqZj94K55G6QSFy8</a>'
             ],
            'Minimum extent agreement': [ee.Image('users/rbfontana/inundation_amazon/AgreementMin'), 1, 10, 'frequency',
'Products agreement on minimum inundation (number of products).'
'<br/><br/>Data acquisition: <a href="https://forms.gle/NqdcANKh6AKihfsn9" target="_blank">https://forms.gle/NqdcANKh6AKihfsn9</a>'
             ]
             },
           
           'Wetland hydrological types':
            {
            ' ': [ee.Image('users/rbfontana/inundation_amazon/Floodplains_Interfluvial'), 1, 2, 'frequency',
'On map: 1 = interfluvial wetlands; 2 = river floodplains.'
"""<br/><br/>Data acquisition: <a href="https://forms.gle/uQSveccumV6vE19k6" target="_blank">Click here</a>"""
             ],
             },
           
           'Regional/local scale':
            {
            'Uatumã (Resende et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/Resende'), 0, 999, 'binary',
'Binary, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 12.5 m | Period: 2006-2011. <br/>'
'Resende et al., 2019. Massive tree mortality from flood pulse disturbances in Amazonian floodplain forests: The collateral effects of hydropower production. '
'Sci. Total Environ. 659, 587–598. https://doi.org/10.1016/j.scitotenv.2018.12.208'
'<br/><br/>Data acquisition: Angélica Resende - <a href="gel.florestal@gmail.com" target="_blank">gel.florestal@gmail.com</a>'
             ],
             'Monte Alegre (Silva et al., 2013)': [ee.Image('users/rbfontana/inundation_amazon/SILVA_LAG_Reclass'), 0, 999, 'binary',
'Binary, Radarsat-1 (C-Band SAR). <br/>'
'Spatial resolution: 500 m | Period: 2001-2014. <br/>'
'Silva et al., 2013. Responses of Aquatic Macrophyte Cover and Productivity to Flooding Variability on the Amazon Floodplain. '
'Glob. Chang. Biol. 19. https://doi.org/10.1111/gcb.12308'
'<br/><br/>Data acquisition: <a href="https://forms.gle/rCFbQJLBZM873LvP6" target="_blank">https://forms.gle/rCFbQJLBZM873LvP6</a>'
            ],
             'Curuai (Silva et al., 2013)': [ee.Image('users/rbfontana/inundation_amazon/SilvaOBI_Reclass'), 0, 999, 'binary',
'Binário, Radarsat-1 (C-Band SAR). <br/>'
'Spatial resolution: 500 m | Period: 2001-2014. <br/>'
'Silva et al., 2013. Responses of Aquatic Macrophyte Cover and Productivity to Flooding Variability on the Amazon Floodplain. '
'Glob. Chang. Biol. 19. https://doi.org/10.1111/gcb.12308'
'<br/><br/>Data acquisition: <a href="https://forms.gle/iCvVkvEd9MjZDra66" target="_blank">https://forms.gle/iCvVkvEd9MjZDra66</a>'
            ],
            'Curuai (Arnesen et al., 2103)': [ee.Image('users/rbfontana/inundation_amazon/Arnesen_FloodFrequency'), 0, 1, 'frequency',
'Flood frequency, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 90 m | Period: 2006-2010. <br/>'
'Arnesen et al., 2013. Monitoring flood extent in the lower Amazon River floodplain using ALOS/PALSAR ScanSAR images. ' 
'Remote Sens. Environ. 130, 51–61. https://doi.org/10.1016/j.rse.2012.10.035'
'<br/><br/>Data acquisition: <a href="https://forms.gle/ibBArDjKnWosK6xr6" target="_blank">https://forms.gle/ibBArDjKnWosK6xr6</a>'
             ],
            'Curuai (Rudorff et al., 2014)': [ee.Image('users/rbfontana/inundation_amazon/Rudorff_Curuai'), 0, 1, 'frequency',
'Flood frequency, LISFLOOD-FP 2D model. <br/>'
'Spatial resolution: 90 m | Period: 2006-2010. <br/>'
'Rudorff et al., 2014. Flooding Dynamics on the Lower Amazon Floodplain: 1. Hydraulic Controls on Water Elevation, Inundation Extent, and River-Floodplain Discharge. ' 
'Water Resour. Res. 50, 619. https://doi.org/10.1002/2013WR014091'
'<br/><br/>Data acquisition: <a href="https://forms.gle/jvW7wfsSTnVoNd71A" target="_blank">https://forms.gle/jvW7wfsSTnVoNd71A</a>'
             ],
            'Mamirauá (Ferreira-Ferreira et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/Mamiraua_FloodDur'), 1, 7, 'frequency',
'Flood frequency, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 12.5 m | Period: 2006-2011. <br/>'
'On map, inundation days per year: <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 1 &emsp; <40 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 2 &emsp; 40-105 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 3 &emsp; 105-125 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 4 &emsp; 125-175 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 5 &emsp; 175-295 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 6 &emsp; 295-365 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 7 &emsp; 365 <br/>'
'Ferreira-Ferreira et al., 2015. Combining ALOS/PALSAR derived vegetation structure and inundation patterns to characterize major vegetation types in the Mamirauá Sustainable Development Reserve, Central Amazon floodplain, Brazil. ' 
'Wetl. Ecol. Manag. 23, 41–59. https://doi.org/10.1007/s11273-014-9359-1'
'<br/><br/>Data acquisition: <a href="https://forms.gle/DPFGivZfGSsPk3rGA" target="_blank">https://forms.gle/DPFGivZfGSsPk3rGA</a>'
             ],
             'Janauacá (Pinel et al., 2019) (SAR)': [ee.Image('users/rbfontana/inundation_amazon/PinelFloodFrequency'), 0, 1, 'frequency',
'Flood frequency, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 30 m | Period: 2006-2011. <br/>'
'Pinel et al., 2019. Flooding dynamics within an Amazonian floodplain: water circulation patterns and inundation duration. '
'Water Resour. Res. 2019WR026081. https://doi.org/10.1029/2019WR026081'
'<br/><br/>Data acquisition: <a href="https://forms.gle/5FPjG3pP5TkVQ8xN8" target="_blank">https://forms.gle/5FPjG3pP5TkVQ8xN8</a>'
             ],
             'Janauacá (Pinel et al., 2019) (Hydro Model)': [ee.Image('users/rbfontana/inundation_amazon/Pinel_Janauaca'), 0, 1, 'frequency',
'Flood frequency, hydrological model. <br/>'
'Spatial resolution: 90 m | Period: 2006-2011. <br/>'
'Pinel et al., 2019. Flooding dynamics within an Amazonian floodplain: water circulation patterns and inundation duration. '
'Water Resour. Res. 2019WR026081. https://doi.org/10.1029/2019WR026081'
'<br/><br/>Data acquisition: <a href="https://forms.gle/xnguZntazDw6vvpWA" target="_blank">https://forms.gle/xnguZntazDw6vvpWA</a>'
             ],
             'Janauacá (Bonnet et al., 2016)': [ee.Image('users/rbfontana/inundation_amazon/Bonnet_Janauaca'), 0, 1, 'frequency',
'Flood frequency, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 30 m | Period: 2006-2011. <br/>'
'Bonnet et al., 2016. Amazonian floodplain water balance based on modelling and analyses of hydrologic and electrical conductivity data. '
'Hydr. Proc. 31, 9, 1702-1718. https://doi.org/10.1002/hyp.11138'
'<br/><br/>Data acquisition: <a href="https://forms.gle/YpzSaMCoDnjQKh4G6" target="_blank">https://forms.gle/YpzSaMCoDnjQKh4G6</a>'
             ],
             'Llanos de Moxos (Ovando et al. 2016)': [ee.Image('users/rbfontana/inundation_amazon/ovandoMODIS'), 0, 1, 'frequency',
'Flood frequency, MODIS (optical imagery). <br/>'
'Spatial resolution: 500 m | Period: 2001-2014. <br/>'
'Ovando et al., 2016. Extreme flood events in the Bolivian Amazon wetlands. '
'J. Hydrol. Reg. Stud. 5, 293–308. https://doi.org/10.1016/j.ejrh.2015.11.004'
'<br/><br/>Data acquisition: <a href="https://forms.gle/ZDykj88yHaouBJrU9" target="_blank">https://forms.gle/ZDykj88yHaouBJrU9</a>'
            ],
             'Pacaya-Samiria (Jensen et al., 2018)': [ee.Image('users/rbfontana/inundation_amazon/ALOS2_Jensen'), 0, 1, 'frequency',
'Flood frequency, ALOS-2 PALSAR-2 (L-band SAR). <br/>'
'Spatial resolution: 50 m | Period: 2014-2018. <br/>'
'Jensen et al., 2018. Assessing L-Band GNSS-Reflectometry and Imaging Radar for Detecting Sub-Canopy Inundation Dynamics in a Tropical Wetlands Complex. '
'Remote Sens. 10, 1431 (2018). https://doi.org/10.3390/rs10091431'
'<br/><br/>Data acquisition: Kat Jensen – <a href="kjensen@ccny.cuny.edu" target="_blank">kjensen@ccny.cuny.edu</a>'
            ],
             'Lower Amazon (Park et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/Park_LowerAmazon'), 0, 1, 'frequency',
'Flood frequency, MODIS (optical imagery). <br/>'
'Spatial resolution: 500 m | Period: 2001-2014. <br/>'
'Park et al., 2019. A Geomorphological Assessment of Wash-Load Sediment Fluxes and Floodplain Sediment Sinks along the Lower Amazon River. '
'Geology 47, 403. https://doi.org/10.1130/G45769.1'
'<br/><br/>Data acquisition: <a href="https://forms.gle/HU5yjtY2u2KuMRA89" target="_blank">https://forms.gle/HU5yjtY2u2KuMRA89</a>'
            ],
             'Lower Amazon (Silva et al., 2013)': [ee.Image(''), 0, 999, 'binary',
'Binary, Radarsat-1 (C-Band SAR). <br/>'
'Spatial resolution: 500 m | Period: 2001-2014. <br/>'
'Silva et al., 2013. Responses of Aquatic Macrophyte Cover and Productivity to Flooding Variability on the Amazon Floodplain. '
'Glob. Chang. Biol. 19. https://doi.org/10.1111/gcb.12308'
'<br/><br/>Data acquisition: <a href='+formlink+' target="_blank">https://forms.gle/AmazonInundationData</a>'
            ],
            },
           
           'Basin scale':
            {
            'THMB (Coe et al., 2007)': [ee.Image('users/rbfontana/inundation_amazon/THMB'), 0.01, 1, 'frequency',
'Flood frequency, hydrological model. <br/>'
'Spatial resolution: 500 m | Period: 1980-2014. <br/>'
'Coe et al., 2007. Simulating the surface waters of the Amazon River basin: impacts of new river geomorphic and flow parameterizations. '
'Hydr. Proc. 22, 14, 2542-2553. https://doi.org/10.1002/hyp.6850'
'<br/><br/>Data acquisition: Michael Coe - <a href="mtcoe@woodwellclimate.org" target="_blank">mtcoe@woodwellclimate.org</a>'
             ],
            'GIEMS-D3 (Aires et al., 2017)': [ee.Image('users/rbfontana/inundation_amazon/GIEMSD3_Amazon_LT'), 0, 1, 'frequency',
'Flood frequency, multiple datasets at high resolution. <br/>'
'Spatial resolution: 90 m | Period: 1993-2007. <br/>'
'Aires et al., 2017. A Global Dynamic Long-Term Inundation Extent Dataset at High Spatial Resolution Derived through Downscaling of Satellite Observations. '
'J. Hydrometeorol. 18, 1305. https://doi.org/10.1175/JHM-D-16-0155.1 '
'<br/><br/>Data acquisition: <a href="https://forms.gle/XnN1DHpmogPrmmda9" target="_blank">https://forms.gle/XnN1DHpmogPrmmda9</a>'
             ],
             'GIEMS-D15 (Fluet-Chouinard et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/giems_amaz21500'), 0, 999, 'binary',
'Binary, fusion of multiple datasets. <br/>'
'Spatial resolution: 232 m | Period: 1993-2004. <br/>'
'Fluet-Chouinard et al., 2015. Development of a global inundation map at high spatial resolution from topographic downscaling of coarse-scale remote sensing data. '
'Remote Sens. Environ. 158, 348–361. https://doi.org/10.1016/j.rse.2014.10.015'
'<br/><br/>Data acquisition: <a href="https://forms.gle/thcGTUyEtSytLJQF6" target="_blank">https://forms.gle/thcGTUyEtSytLJQF6</a>'
             ],
            'GIEMS-2 (Prigent et al., 2020)': [ee.Image('users/rbfontana/inundation_amazon/GIEMS-2'), 0, 1, 'frequency',
'Flood frequency, multiple datasets at coarse resolution. <br/>'
'Spatial resolution: 25 km | Period: 1992-2015. <br/>'
'Prigent et al., 2020. Satellite-Derived Global Surface Water Extent and Dynamics Over the Last 25 Years (GIEMS-2). '
'J. Geophys. Res. Atmos. 125, 1 (2020). https://doi.org/10.1029/2019JD030711 '
'<br/><br/>Data acquisition: <a href="https://forms.gle/3eSaSCZ4DRyNMLVz7" target="_blank">https://forms.gle/3eSaSCZ4DRyNMLVz7</a>'
             ],    
            'CaMa-Flood (Yamazaki et al., 2011)': [ee.Image('users/rbfontana/inundation_amazon/CaMA-Flood'), 0, 100, 'frequency',
'Flood frequency, hydrological model. <br/>'
'Spatial resolution: 500 m | Period: 1980-2014. <br/>'
'Yamazaki et al., 2011. A Physically Based Description of Floodplain Inundation Dynamics in a Global River Routing Model. '
'Water Resour. Res. 47, 1. https://doi.org/10.1029/2010WR009726 '
'<br/><br/>Data acquisition: <a href="http://hydro.iis.u-tokyo.ac.jp/~yamadai/cama-flood/" target="_blank">http://hydro.iis.u-tokyo.ac.jp/~yamadai/cama-flood/</a>'
             ],   
            'ESA-CCI (Bontemps et al., 2013)': [ee.Image('users/rbfontana/inundation_amazon/esa20101500'), 150, 999, 'binary',
'Binary, multisatellite (optical and SAR). <br/>'
'Spatial resolution: 300 m | Period: 1992-2015. <br/>'
'Bontemps et al., 2013. Consistent global land cover maps for climate modelling communities: current achievements of the ESA’s land cover CCI. '
'Proceedings of the ESA Living Planet Symposium. Edinburgh.'
'<br/><br/>Data acquisition: <a href="https://climate.esa.int/" target="_blank">https://climate.esa.int/</a>'
             ],
             'GLWD (Lehner & Döll, 2004)': [ee.Image('users/rbfontana/inundation_amazon/glwd_Amaz500'), 0, 999, 'binary',
'Binary, fusion of multiple datasets. <br/>'
'Spatial resolution: 1 km | Period: 1992-2004. <br/>'
'Lehner & Döll, 2004. Development and validation of a global database of lakes, reservoirs and wetlands. '
'J. Hydrol. 296, 1–22. https://doi.org/10.1016/j.jhydrol.2004.03.028'
'<br/><br/>Data acquisition: <a href="https://www.worldwildlife.org/pages/global-lakes-and-wetlands-database" target="_blank">https://www.worldwildlife.org/pages/global-lakes-and-wetlands-database</a>'
             ],
             'CIFOR (Gumbricht et al., 2017) ': [ee.Image('users/rbfontana/inundation_amazon/cifor_amaz500'), 0, 999, 'binary',
'Binary, fusion of multiple datasets. <br/>'
'Spatial resolution: 232 m | Period: 1950-2000. <br/>'
'Gumbricht et al., 2017. An expert system model for mapping tropical wetlands and peatlands reveals South America as the largest contributor. '
'Glob. Chang. Biol. 23, 3581–3599. https://doi.org/10.1111/gcb.13689'
'<br/><br/>Data acquisition: <a href="https://www2.cifor.org/global-wetlands/" target="_blank">https://www2.cifor.org/global-wetlands/</a>'
             ],
             'MGB (Siqueira et al., 2018)': [ee.Image('users/rbfontana/inundation_amazon/MGB_AmazonBasin'), 0, 1, 'frequency',
'Flood frequency, hydrological model. <br/>'
'Spatial resolution: 500 m | Period: 1980-2014. <br/>'
'Siqueira et al., 2018. Toward continental hydrologic–hydrodynamic modeling in South America. '
'Hydrol. Earth Syst. Sci. 22, 4815–4842. https://doi.org/10.5194/hess-22-4815-2018'
'<br/><br/>Data acquisition: <a href="https://forms.gle/GCHdn9yohXGKK5ti6" target="_blank">https://forms.gle/GCHdn9yohXGKK5ti6</a>'
             ],
             'G3WBM (Yamazaki et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/g3wbm_amaz500'), 0, 999, 'binary',
'Binary, Landsat optical imagery. <br/>'
'Spatial resolution: 90 m | Period: 1990-2010. <br/>'
'Yamazaki et al., 2015. Development of a global ~90m water body map using multi-temporal Landsat images. '
'Remote Sens. Environ. 171, 337-351. https://doi.org/10.1016/j.rse.2015.10.014'
'<br/><br/>Data acquisition: <a href="http://hydro.iis.u-tokyo.ac.jp/~yamadai/G3WBM/" target="_blank">http://hydro.iis.u-tokyo.ac.jp/~yamadai/G3WBM/</a>'
             ],
             'GSW (Pekel et al., 2016)': [ee.Image("JRC/GSW1_3/GlobalSurfaceWater").select('occurrence'), 0, 100, 'frequency',
'Flood frequency, Landsat optical imagery. <br/>'
'Spatial resolution: 30 m | Period: 1980-2014. <br/>'
'Pekel et al., 2016. High-resolution mapping of global surface water and its long-term changes. '
'Nature 540, 418–422. https://doi.org/10.1038/nature20584'
'<br/><br/>Data acquisition: <a href="https://global-surface-water.appspot.com/" target="_blank">https://global-surface-water.appspot.com/</a>'
             ],
            'Chapman (Chapman et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/chapman_amaz_compres500'), 0, 4, 'binary',
'Binary, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 90 m | Period: 2006-2011. <br/>'
'Chapman et al., 2015. Mapping Regional Inundation with Spaceborne L-Band SAR. '
'Remote Sens. 7(5), 5440-5470. https://doi.org/10.3390/rs70505440'
'<br/><br/>Data acquisition: <a href="https://asf.alaska.edu/data-sets/derived-data-sets/wetlands-measures/" target="_blank">https://asf.alaska.edu/data-sets/derived-data-sets/wetlands-measures/</a>'
            ],
            'Hess (Hess et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/hess_wetl_mask_compressed500'), 0, 999, 'binary',
'Binary, JERS-1 (L-band SAR). <br/>'
'Spatial resolution: 90 m | Period: 1995-1996. <br/>'
'Hess et al., 2015. Wetlands of the Lowland Amazon Basin: Extent, Vegetative Cover, and Dual-season Inundated Area as Mapped with JERS-1 Synthetic Aperture Radar. '
'Wetlands 35, 745–756. https://doi.org/10.1007/s13157-015-0666-y'
'<br/><br/>Data acquisition: <a href="https://daac.ornl.gov/LBA/guides/LC07_Amazon_Wetlands.html" target="_blank">https://daac.ornl.gov/LBA/guides/LC07_Amazon_Wetlands.html</a>'
            ],
            'SWAF-HR (Parrens et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/SWAF_HR'), 0, 1, 'frequency',
'Flood frequency, SMOS-based product (L-band passive microwave). <br/>'
'Spatial resolution: 1 km | Period: 2010-2016. <br/>'
'Parrens et al., 2019. High Resolution Mapping of Inundation Area in the Amazon Basin from a Combination of L-Band Passive Microwave, Optical and Radar Datasets. '
'Int. J. Appl. Earth Obs. Geoinf. 81, 58. https://doi.org/10.1016/j.jag.2019.04.011'
'<br/><br/>Data acquisition: <a href="https://forms.gle/HeHi5npQb5WzvhcH6" target="_blank">https://forms.gle/HeHi5npQb5WzvhcH6</a>'
            ],
            'Rosenqvist (Rosenqvist et al., 2020)': [ee.Image('users/rbfontana/inundation_amazon/RosenqvistMax'), 0, 999, 'binary',
'Binary, ALOS-2 PALSAR-2 (L-band SAR). <br/>'
'Spatial resolution: 50 m | Period: 2014-2018. <br/>'
'Rosenqvist et al., 2020. Mapping of Maximum and Minimum Inundation Extents in the Amazon Basin 2014–2017 with ALOS-2 PALSAR-2 ScanSAR Time-Series Data. '
'Remote Sens. 12, 1326. https://doi.org/10.3390/rs12081326'
'<br/><br/>Data acquisition: Ake Rosenqvist - <a href="ake.rosenqvist@soloeo.com" target="_blank">ake.rosenqvist@soloeo.com</a>'
            ],
            }
        }

btnMap = w.Button(description='Select map to display: ', layout=btnLayout, style=btnStyle)

mapDwn = w.widgets.Dropdown(options=mapList[catW.value], description='Product: ', layout=w.Layout(height='40px', width = '75%'))
def update_mapDwn(cat):
    mapDwn.options = mapList[cat.new]
catW.observe(update_mapDwn, 'value')
    
input_ops = w.interactive(show_map, map2show=mapDwn)

map_description = w.HTML(mapDwn.value[-1])
def update_map_description(product):
    map_description.value = product.new[-1]
mapDwn.observe(update_map_description, 'value')

btnDescription = w.Button(description='Product description: ', layout=btnLayout, style=btnStyle)
box_description = w.VBox([btnDescription, map_description])
#box_description = w.Accordion([ map_description])

btnAbout = w.Button(description='About the platform: ', layout=btnLayout, style=btnStyle)
about.layout.width = elements_width
box_about = w.VBox([w.HTML('<br/>'), btnAbout, about])

btnContact = w.Button(description='Contact: ', layout=btnLayout, style=btnStyle)
box_contact = w.VBox([w.HTML('<br/>'), btnContact, w.HTML('rbarbedofontana@gmail.com <br/> ayan.fleischmann@gmail.com')])

btnLogos = w.Button(description='Support: ', layout=btnLayout, style=btnStyle)
box_logos = w.VBox([w.HTML('<br/>'), btnLogos, logos])


In [5]:
# PT version

# Descricao
subtitulo = w.HTML(value='<{size}>Exibindo a extensão de inundação da maior bacia hidrográfica da Terra <br/> Plataforma WebGIS | Versão 0.1</{size}>'.format(size='h3'))
descricao = w.HTML(value= 
'<h4 align="justify">O aplicativo WebGIS Amazon-Inundation permite a visualização da extensão de inundação em toda a bacia do rio Amazonas '
         'com base em 30 conjuntos de dados que foram desenvolvidos nas últimas décadas com diferentes resoluções espaciais e temporais. </h4>'
                     '<p style="font-size:12px;">Desenvolvido por Rafael Barbedo e Ayan Fleischmann <br/>'
                     'Instituto de Pesquisas Hidráulicas, Universidade Federal do Rio Grande do Sul</p>')
descricao.layout.width = elements_width

sobre = w.HTML('<{size} align="justify">A bacia amazônica é a maior da Terra, e abriga ecossistemas de água doce únicos dos quais milhares de pessoas dependem para seus meios de subsistência. '
                     'Múltiplos ecossistemas são adaptados à dinâmica sazonal de inundação dos rios amazônicos. <br/>'
                     
'O aplicativo WebSIG “Amazon-Inundation” fornece uma maneira fácil de visualizar a extensão da inundação em toda a bacia a partir de múltiplos produtos que têm sido desenvolvidos nas últimas décadas. <br/>'
                     
'Os produtos têm diferentes fontes de dados, de micro-ondas passivas e radar de abertura sintética a imagens ópticas e modelagem hidrológica, com resolução espacial variando de 12.5 m a 25 km. <br/>'

'Quatro tipos de produtos estão disponíveis: <br/>'
               '- Dois mapas de concordância com 1 km de resolução espacial, mostrando, para cada pixel de 1 km, '
                   'quantos conjuntos de dados concordam que este é inundado considerando os mapas de inundação máximos ou mínimos. <br/>'
               '- Tipos de áreas úmidas (1- áreas úmidas interfluviais e pequenas planícies de inundação; 2- planícies de inundação médias a grandes). <br/>'
               '- Escala regional/local: produtos desenvolvidos para sub-regiões específicas da Amazônia. <br/>'
               '- Escala da bacia: produtos desenvolvidos para escala global ou especialmente para a bacia hidrográfica do rio Amazonas. <br/><br/>'

'Referências e disponibilidade de dados são fornecidos para cada conjunto de dados. Para mais detalhes, e para citar esta ferramenta, consulte: <br/><br/>'

'Fleischmann et al., 2021, How much inundation occurs in the Amazon River basin? '
'<a href="https://www.essoar.org/doi/10.1002/essoar.10508718.1" target="_blank">https://forms.gle/HcqZj94K55G6QSFy8</a></{size}>'
                     .format(size='p'))


# Categorias
catLista = ['Concordância entre as bases de dados a 1 km', 'Tipos de áreas úmidas', 'Escala regional/local', 'Escala de bacia']
catWpt = w.widgets.RadioButtons(options=catLista, description='Categoria: ', layout=w.Layout(width='400px'))

# Mapas
mapLista = {'Concordância entre as bases de dados a 1 km':
            {
            'Concordância na máxima extensão': [ee.Image('users/rbfontana/inundation_amazon/AgreementMax'), 1, 14, 'frequency',
'Concordância dos produtos na máxima extensão (número de produtos).'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/HcqZj94K55G6QSFy8" target="_blank">https://forms.gle/HcqZj94K55G6QSFy8</a>'
             ],
            'Concordância na mínima extensão': [ee.Image('users/rbfontana/inundation_amazon/AgreementMin'), 1, 10, 'frequency',
'Concordância dos produtos na mínima extensão (número de produtos).'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/NqdcANKh6AKihfsn9" target="_blank">https://forms.gle/NqdcANKh6AKihfsn9</a>'
             ]
             },
           
           'Tipos de áreas úmidas':
            {
            ' ': [ee.Image('users/rbfontana/inundation_amazon/Floodplains_Interfluvial'), 1, 2, 'frequency',
'No mapa: 1- áreas úmidas interfluviais e pequenas planícies de inundação; 2- planícies de inundação médias a grandes.'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/uQSveccumV6vE19k6" target="_blank">https://forms.gle/uQSveccumV6vE19k6</a>'
             ],
             },
           
           'Escala regional/local':
            {
            'Uatumã (Resende et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/Resende'), 0, 999, 'binary',
'Binário, ALOS-PALSAR (L-band SAR). <br/>'
'Resolução espacial: 12.5 m | Período: 2006-2011. <br/>'
'Resende et al., 2019. Massive tree mortality from flood pulse disturbances in Amazonian floodplain forests: The collateral effects of hydropower production. '
'Sci. Total Environ. 659, 587–598. https://doi.org/10.1016/j.scitotenv.2018.12.208'
'<br/><br/>Aquisição dos dados: Angélica Resende - <a href="gel.florestal@gmail.com" target="_blank">gel.florestal@gmail.com</a>'
             ],
             'Monte Alegre (Silva et al., 2013)': [ee.Image('users/rbfontana/inundation_amazon/SILVA_LAG_Reclass'), 0, 999, 'binary',
'Binário, Radarsat-1 (C-Band SAR). <br/>'
'Resolução espacial: 500 m | Período: 2001-2014. <br/>'
'Silva et al., 2013. Responses of Aquatic Macrophyte Cover and Productivity to Flooding Variability on the Amazon Floodplain. '
'Glob. Chang. Biol. 19. https://doi.org/10.1111/gcb.12308'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/rCFbQJLBZM873LvP6" target="_blank">https://forms.gle/rCFbQJLBZM873LvP6</a>'
            ],
             'Curuai (Silva et al., 2013)': [ee.Image('users/rbfontana/inundation_amazon/SilvaOBI_Reclass'), 0, 999, 'binary',
'Binário, Radarsat-1 (C-Band SAR). <br/>'
'Resolução espacial: 500 m | Período: 2001-2014. <br/>'
'Silva et al., 2013. Responses of Aquatic Macrophyte Cover and Productivity to Flooding Variability on the Amazon Floodplain. '
'Glob. Chang. Biol. 19. https://doi.org/10.1111/gcb.12308'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/iCvVkvEd9MjZDra66" target="_blank">https://forms.gle/iCvVkvEd9MjZDra66</a>'
            ],
            'Curuai (Arnesen et al., 2103)': [ee.Image('users/rbfontana/inundation_amazon/Arnesen_FloodFrequency'), 0, 1, 'frequency',
'Frequência de inundação, ALOS-PALSAR (L-band SAR). <br/>'
'Resolução espacial: 90 m | Período: 2006-2010. <br/>'
'Arnesen et al., 2013. Monitoring flood extent in the lower Amazon River floodplain using ALOS/PALSAR ScanSAR images. ' 
'Remote Sens. Environ. 130, 51–61. https://doi.org/10.1016/j.rse.2012.10.035'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/ibBArDjKnWosK6xr6" target="_blank">https://forms.gle/ibBArDjKnWosK6xr6</a>'
             ],
            'Curuai (Rudorff et al., 2014)': [ee.Image('users/rbfontana/inundation_amazon/Rudorff_Curuai'), 0, 1, 'frequency',
'Frequência de inundação, LISFLOOD-FP 2D model. <br/>'
'Resolução espacial: 90 m | Período: 2006-2010. <br/>'
'Rudorff et al., 2014. Flooding Dynamics on the Lower Amazon Floodplain: 1. Hydraulic Controls on Water Elevation, Inundation Extent, and River-Floodplain Discharge. ' 
'Water Resour. Res. 50, 619. https://doi.org/10.1002/2013WR014091'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/jvW7wfsSTnVoNd71A" target="_blank">https://forms.gle/jvW7wfsSTnVoNd71A</a>'
             ],
            'Mamirauá (Ferreira-Ferreira et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/Mamiraua_FloodDur'), 1, 7, 'frequency',
'Frequência de inundação, ALOS-PALSAR (L-band SAR). <br/>'
'Resolução espacial: 12.5 m | Período: 2006-2011. <br/>'
'No mapa, dias de inundação por ano: <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 1 &emsp; <40 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 2 &emsp; 40-105 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 3 &emsp; 105-125 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 4 &emsp; 125-175 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 5 &emsp; 175-295 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 6 &emsp; 295-365 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 7 &emsp; 365 <br/>'
'Ferreira-Ferreira et al., 2015. Combining ALOS/PALSAR derived vegetation structure and inundation patterns to characterize major vegetation types in the Mamirauá Sustainable Development Reserve, Central Amazon floodplain, Brazil. ' 
'Wetl. Ecol. Manag. 23, 41–59. https://doi.org/10.1007/s11273-014-9359-1'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/DPFGivZfGSsPk3rGA" target="_blank">https://forms.gle/DPFGivZfGSsPk3rGA</a>'
             ],
             'Janauacá (Pinel et al., 2019) (SAR)': [ee.Image('users/rbfontana/inundation_amazon/PinelFloodFrequency'), 0, 1, 'frequency',
'Frequência de inundação, ALOS-PALSAR (L-band SAR). <br/>'
'Resolução espacial: 30 m | Período: 2006-2011. <br/>'
'Pinel et al., 2019. Flooding dynamics within an Amazonian floodplain: water circulation patterns and inundation duration. '
'Water Resour. Res. 2019WR026081. https://doi.org/10.1029/2019WR026081'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/5FPjG3pP5TkVQ8xN8" target="_blank">https://forms.gle/5FPjG3pP5TkVQ8xN8</a>'
             ],
             'Janauacá (Pinel et al., 2019) (Hydro Model)': [ee.Image('users/rbfontana/inundation_amazon/Pinel_Janauaca'), 0, 1, 'frequency',
'Frequência de inundação, hydrological model. <br/>'
'Resolução espacial: 90 m | Período: 2006-2011. <br/>'
'Pinel et al., 2019. Flooding dynamics within an Amazonian floodplain: water circulation patterns and inundation duration. '
'Water Resour. Res. 2019WR026081. https://doi.org/10.1029/2019WR026081'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/xnguZntazDw6vvpWA" target="_blank">https://forms.gle/xnguZntazDw6vvpWA</a>'
             ],
             'Janauacá (Bonnet et al., 2016)': [ee.Image('users/rbfontana/inundation_amazon/Bonnet_Janauaca'), 0, 1, 'frequency',
'Flood frequency, ALOS-PALSAR (L-band SAR). <br/>'
'Resolução espacial: 30 m | Período: 2006-2011. <br/>'
'Bonnet et al., 2016. Amazonian floodplain water balance based on modelling and analyses of hydrologic and electrical conductivity data. '
'Hydr. Proc. 31, 9, 1702-1718. https://doi.org/10.1002/hyp.11138'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/YpzSaMCoDnjQKh4G6" target="_blank">https://forms.gle/YpzSaMCoDnjQKh4G6</a>'
             ],
             'Llanos de Moxos (Ovando et al. 2016)': [ee.Image('users/rbfontana/inundation_amazon/ovandoMODIS'), 0, 1, 'frequency',
'Frequência de inundação, MODIS (optical imagery). <br/>'
'Resolução espacial: 500 m | Período: 2001-2014. <br/>'
'Ovando et al., 2016. Extreme flood events in the Bolivian Amazon wetlands. '
'J. Hydrol. Reg. Stud. 5, 293–308. https://doi.org/10.1016/j.ejrh.2015.11.004'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/ZDykj88yHaouBJrU9" target="_blank">https://forms.gle/ZDykj88yHaouBJrU9</a>'
            ],
             'Pacaya-Samiria (Jensen et al., 2018)': [ee.Image('users/rbfontana/inundation_amazon/ALOS2_Jensen'), 0, 1, 'frequency',
'Frequência de inundação, ALOS-2 PALSAR-2 (L-band SAR). <br/>'
'Resolução espacial: 50 m | Período: 2014-2018. <br/>'
'Jensen et al., 2018. Assessing L-Band GNSS-Reflectometry and Imaging Radar for Detecting Sub-Canopy Inundation Dynamics in a Tropical Wetlands Complex. '
'Remote Sens. 10, 1431 (2018). https://doi.org/10.3390/rs10091431'
'<br/><br/>Aquisição dos dados: Kat Jensen – <a href="kjensen@ccny.cuny.edu" target="_blank">kjensen@ccny.cuny.edu</a>'
            ],
             'Lower Amazon (Park et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/Park_LowerAmazon'), 0, 1, 'frequency',
'Flood frequency, MODIS (optical imagery). <br/>'
'Resolução espacial: 500 m | Período: 2001-2014. <br/>'
'Park et al., 2019. A Geomorphological Assessment of Wash-Load Sediment Fluxes and Floodplain Sediment Sinks along the Lower Amazon River. '
'Geology 47, 403. https://doi.org/10.1130/G45769.1'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/HU5yjtY2u2KuMRA89" target="_blank">https://forms.gle/HU5yjtY2u2KuMRA89</a>'
            ],
            },
           
           'Escala de bacia':
            {
            'THMB (Coe et al., 2007)': [ee.Image('users/rbfontana/inundation_amazon/THMB'), 0.01, 1, 'frequency',
'Frequência de inundação, hydrological model. <br/>'
'Resolução espacial: 500 m | Período: 1980-2014. <br/>'
'Coe et al., 2007. Simulating the surface waters of the Amazon River basin: impacts of new river geomorphic and flow parameterizations. '
'Hydr. Proc. 22, 14, 2542-2553. https://doi.org/10.1002/hyp.6850'
'<br/><br/>Aquisição dos dados: Michael Coe - <a href="mtcoe@woodwellclimate.org" target="_blank">mtcoe@woodwellclimate.org</a>'
             ],
            'GIEMS-D3 (Aires et al., 2017)': [ee.Image('users/rbfontana/inundation_amazon/GIEMSD3_Amazon_LT'), 0, 1, 'frequency',
'Frequência de inundação, multiple datasets at high resolution. <br/>'
'Resolução espacial: 90 m | Período: 1993-2007. <br/>'
'Aires et al., 2017. A Global Dynamic Long-Term Inundation Extent Dataset at High Spatial Resolution Derived through Downscaling of Satellite Observations. '
'J. Hydrometeorol. 18, 1305. https://doi.org/10.1175/JHM-D-16-0155.1 '
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/XnN1DHpmogPrmmda9" target="_blank">https://forms.gle/XnN1DHpmogPrmmda9</a>'
             ],
             'GIEMS-D15 (Fluet-Chouinard et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/giems_amaz21500'), 0, 999, 'binary',
'Binário, fusion of multiple datasets. <br/>'
'Resolução espacial: 232 m | Período: 1993-2004. <br/>'
'Fluet-Chouinard et al., 2015. Development of a global inundation map at high spatial resolution from topographic downscaling of coarse-scale remote sensing data. '
'Remote Sens. Environ. 158, 348–361. https://doi.org/10.1016/j.rse.2014.10.015'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/thcGTUyEtSytLJQF6" target="_blank">https://forms.gle/thcGTUyEtSytLJQF6</a>'
             ],
            'GIEMS-2 (Prigent et al., 2020)': [ee.Image('users/rbfontana/inundation_amazon/GIEMS-2'), 0, 1, 'frequency',
'Frequência de inundação, multiple datasets at coarse resolution. <br/>'
'Resolução espacial: 25 km | Período: 1992-2015. <br/>'
'Prigent et al., 2020. Satellite-Derived Global Surface Water Extent and Dynamics Over the Last 25 Years (GIEMS-2). '
'J. Geophys. Res. Atmos. 125, 1 (2020). https://doi.org/10.1029/2019JD030711 '
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/3eSaSCZ4DRyNMLVz7" target="_blank">https://forms.gle/3eSaSCZ4DRyNMLVz7</a>'
             ],    
            'CaMa-Flood (Yamazaki et al., 2011)': [ee.Image('users/rbfontana/inundation_amazon/CaMA-Flood'), 0, 100, 'frequency',
'Frequência de inundação, hydrological model. <br/>'
'Resolução espacial: 500 m | Período: 1980-2014. <br/>'
'Yamazaki et al., 2011. A Physically Based Description of Floodplain Inundation Dynamics in a Global River Routing Model. '
'Water Resour. Res. 47, 1. https://doi.org/10.1029/2010WR009726 '
'<br/><br/>Aquisição dos dados: <a href="http://hydro.iis.u-tokyo.ac.jp/~yamadai/cama-flood/" target="_blank">http://hydro.iis.u-tokyo.ac.jp/~yamadai/cama-flood/</a>'
             ],   
            'ESA-CCI (Bontemps et al., 2013)': [ee.Image('users/rbfontana/inundation_amazon/esa20101500'), 150, 999, 'binary',
'Binário, multisatellite (optical and SAR). <br/>'
'Resolução espacial: 300 m | Período: 1992-2015. <br/>'
'Bontemps et al., 2013. Consistent global land cover maps for climate modelling communities: current achievements of the ESA’s land cover CCI. '
'Proceedings of the ESA Living Planet Symposium. Edinburgh.'
'<br/><br/>Aquisição dos dados: <a href="https://climate.esa.int/" target="_blank">https://climate.esa.int/</a>'
             ],
             'GLWD (Lehner & Döll, 2004)': [ee.Image('users/rbfontana/inundation_amazon/glwd_Amaz500'), 0, 999, 'binary',
'Binário, fusion of multiple datasets. <br/>'
'Resolução espacial: 1 km | Período: 1992-2004. <br/>'
'Lehner & Döll, 2004. Development and validation of a global database of lakes, reservoirs and wetlands. '
'J. Hydrol. 296, 1–22. https://doi.org/10.1016/j.jhydrol.2004.03.028'
'<br/><br/>Aquisição dos dados: <a href="https://www.worldwildlife.org/pages/global-lakes-and-wetlands-database" target="_blank">https://www.worldwildlife.org/pages/global-lakes-and-wetlands-database</a>'
             ],
             'CIFOR (Gumbricht et al., 2017) ': [ee.Image('users/rbfontana/inundation_amazon/cifor_amaz500'), 0, 999, 'binary',
'Binário, fusion of multiple datasets. <br/>'
'Resolução espacial: 232 m | Período: 1950-2000. <br/>'
'Gumbricht et al., 2017. An expert system model for mapping tropical wetlands and peatlands reveals South America as the largest contributor. '
'Glob. Chang. Biol. 23, 3581–3599. https://doi.org/10.1111/gcb.13689'
'<br/><br/>Aquisição dos dados: <a href="https://www2.cifor.org/global-wetlands/" target="_blank">https://www2.cifor.org/global-wetlands/</a>'
             ],
             'MGB (Siqueira et al., 2018)': [ee.Image('users/rbfontana/inundation_amazon/MGB_AmazonBasin'), 0, 1, 'frequency',
'Frequência de inundação, hydrological model. <br/>'
'Resolução espacial: 500 m | Período: 1980-2014. <br/>'
'Siqueira et al., 2018. Toward continental hydrologic–hydrodynamic modeling in South America. '
'Hydrol. Earth Syst. Sci. 22, 4815–4842. https://doi.org/10.5194/hess-22-4815-2018'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/GCHdn9yohXGKK5ti6" target="_blank">https://forms.gle/GCHdn9yohXGKK5ti6</a>'
             ],
             'G3WBM (Yamazaki et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/g3wbm_amaz500'), 0, 999, 'binary',
'Binário, Landsat optical imagery. <br/>'
'Resolução espacial: 90 m | Período: 1990-2010. <br/>'
'Yamazaki et al., 2015. Development of a global ~90m water body map using multi-temporal Landsat images. '
'Remote Sens. Environ. 171, 337-351. https://doi.org/10.1016/j.rse.2015.10.014'
'<br/><br/>Data acquisition: <a href="http://hydro.iis.u-tokyo.ac.jp/~yamadai/G3WBM/" target="_blank">http://hydro.iis.u-tokyo.ac.jp/~yamadai/G3WBM/</a>'
             ],
             'GSW (Pekel et al., 2016)': [ee.Image("JRC/GSW1_3/GlobalSurfaceWater").select('occurrence'), 0, 100, 'frequency',
'Frequência de inundação, Landsat optical imagery. <br/>'
'Resolução espacial: 30 m | Período: 1980-2014. <br/>'
'Pekel et al., 2016. High-resolution mapping of global surface water and its long-term changes. '
'Nature 540, 418–422. https://doi.org/10.1038/nature20584'
'<br/><br/>Aquisição dos dados: <a href="https://global-surface-water.appspot.com/" target="_blank">https://global-surface-water.appspot.com/</a>'
             ],
            'Chapman (Chapman et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/chapman_amaz_compres500'), 0, 4, 'binary',
'Binário, ALOS-PALSAR (L-band SAR). <br/>'
'Resolução espacial: 90 m | Período: 2006-2011. <br/>'
'Chapman et al., 2015. Mapping Regional Inundation with Spaceborne L-Band SAR. '
'Remote Sens. 7(5), 5440-5470. https://doi.org/10.3390/rs70505440'
'<br/><br/>Aquisição dos dados: <a href="https://asf.alaska.edu/data-sets/derived-data-sets/wetlands-measures/" target="_blank">https://asf.alaska.edu/data-sets/derived-data-sets/wetlands-measures/</a>'
            ],
            'Hess (Hess et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/hess_wetl_mask_compressed500'), 0, 999, 'binary',
'Binário, JERS-1 (L-band SAR). <br/>'
'Resolução espacial: 90 m | Período: 1995-1996. <br/>'
'Hess et al., 2015. Wetlands of the Lowland Amazon Basin: Extent, Vegetative Cover, and Dual-season Inundated Area as Mapped with JERS-1 Synthetic Aperture Radar. '
'Wetlands 35, 745–756. https://doi.org/10.1007/s13157-015-0666-y'
'<br/><br/>Aquisição dos dados: <a href="https://daac.ornl.gov/LBA/guides/LC07_Amazon_Wetlands.html" target="_blank">https://daac.ornl.gov/LBA/guides/LC07_Amazon_Wetlands.html</a>'
            ],
            'SWAF-HR (Parrens et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/SWAF_HR'), 0, 1, 'frequency',
'Frequência de inundação, SMOS-based product (L-band passive microwave). <br/>'
'Resolução espacial: 1 km | Período: 2010-2016. <br/>'
'Parrens et al., 2019. High Resolution Mapping of Inundation Area in the Amazon Basin from a Combination of L-Band Passive Microwave, Optical and Radar Datasets. '
'Int. J. Appl. Earth Obs. Geoinf. 81, 58. https://doi.org/10.1016/j.jag.2019.04.011'
'<br/><br/>Aquisição dos dados: <a href="https://forms.gle/HeHi5npQb5WzvhcH6" target="_blank">https://forms.gle/HeHi5npQb5WzvhcH6</a>'
            ],
            'Rosenqvist (Rosenqvist et al., 2020)': [ee.Image('users/rbfontana/inundation_amazon/RosenqvistMax'), 0, 999, 'binary',
'Binário, ALOS-2 PALSAR-2 (L-band SAR). <br/>'
'Resolução espacial: 50 m | Período: 2014-2018. <br/>'
'Rosenqvist et al., 2020. Mapping of Maximum and Minimum Inundation Extents in the Amazon Basin 2014–2017 with ALOS-2 PALSAR-2 ScanSAR Time-Series Data. '
'Remote Sens. 12, 1326. https://doi.org/10.3390/rs12081326'
'<br/><br/>Aquisição dos dados: Ake Rosenqvist - <a href="ake.rosenqvist@soloeo.com" target="_blank">ake.rosenqvist@soloeo.com</a>'
            ],
            }
        }

btnMapa = w.Button(description='Selecione o mapa para mostrar: ', layout=btnLayout, style=btnStyle)

mapDwnPt = w.widgets.Dropdown(options=mapLista[catWpt.value], description='Produto: ', layout=w.Layout(height='40px', width = '75%'))
def atualiza_mapDwn(cat):
    mapDwnPt.options = mapLista[cat.new]
catWpt.observe(atualiza_mapDwn, 'value')
    
input_ops_pt = w.interactive(show_map, map2show=mapDwnPt)

mapa_descricao = w.HTML(mapDwnPt.value[-1])
def atualiza_descricao(product):
    mapa_descricao.value = product.new[-1]
mapDwnPt.observe(atualiza_descricao, 'value')

btnDescricao = w.Button(description='Descrição do produto: ', layout=btnLayout, style=btnStyle)
box_descricao = w.VBox([btnDescricao, mapa_descricao])

btnSobre = w.Button(description='Sobre a plataforma: ', layout=btnLayout, style=btnStyle)
about.layout.width = elements_width
box_sobre = w.VBox([w.HTML('<br/>'), btnSobre, sobre])

btnContato = w.Button(description='Contato: ', layout=btnLayout, style=btnStyle)
box_contato = w.VBox([w.HTML('<br/>'), btnContato, w.HTML('rbarbedofontana@gmail.com <br/> ayan.fleischmann@gmail.com')])

btnLogosPt = w.Button(description='Suporte: ', layout=btnLayout, style=btnStyle)
box_logos_pt = w.VBox([w.HTML('<br/>'), btnLogosPt, logos])


In [6]:
# Final front
carousel_layout = w.Layout(#overflow='scroll hidden',
                    #border='3px solid black',
                    width='100%',
                    height='100%',
                    flex_flow='column',
                    display='flex-shrink')

carousel = w.Box(children=[title, subtitle, photo, description, btnMap, catW, input_ops, box_description, box_about, box_contact, box_logos], layout=carousel_layout)
carousel.layout.display = 'block' #Blocks the display visualization so items wont be hidden
carouselpt = w.Box(children=[title, subtitulo, photo, descricao, btnMapa, catWpt, input_ops_pt, box_descricao, box_sobre, box_contato, box_logos_pt], layout=carousel_layout)
carouselpt.layout.display = 'block' #Blocks the display visualization so items wont be hidden

envelope_layout = w.Layout(#overflow='scroll hidden',
                    #border='3px solid black',
                    width='36%',
                    height='100%',
                    flex_flow='column',
                    display='flex-shrink')
tab = w.Tab(children = [carousel, carouselpt])
tab.set_title(0, 'English')
tab.set_title(1, 'Português')
envelope = w.Box(children=[tab], layout=envelope_layout)
# tab.layout.width = '625px'

box_layout = w.Layout(
    height='94vh',  # viewport height
    width='100%',
    grid_gap='5px',
    overflow='auto',
    align_items='stretch',
    margin='0px',
    border='0px',
    padding='0px',
)
box = w.HBox([envelope, Map], layout=box_layout)

display(box)
#Map